# Deploying SVM Model

## Step 1: Import Libraries

In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import segyio

## Step 2: Load and Prepare Data

In [2]:
# Note: the the xline header info location is at segyio.su.cdpx [181]
def segy2numpy(filename: str) -> np.array:
    with segyio.open(filename, xline=181) as segyfile:
        return segyio.tools.cube(segyfile)

In [3]:
# First load datasets
filename_pp = "C:\\Users\\hsk21\\Desktop\\MLcourse\\github\\Seismic-Fault-Detection-using-Convolutional-Neural-Network\\Equinor Synthetic model\\Equinor Synthetic model\\issap20_Pp.sgy"
filename_ai = "C:\\Users\\hsk21\\Desktop\\MLcourse\\github\\Seismic-Fault-Detection-using-Convolutional-Neural-Network\\Equinor Synthetic model\\Equinor Synthetic model\\issap20_Fault.sgy"
filename_fault = "C:\\Users\\hsk21\\Desktop\\MLcourse\\github\\Seismic-Fault-Detection-using-Convolutional-Neural-Network\\Equinor Synthetic model\\Equinor Synthetic model\\issap20_Fault.sgy"


In [4]:
# Note: the the xline header info location is at segyio.su.cdpx [181]
def segy2numpy(filename: str) -> np.array:
    with segyio.open(filename, xline=181) as segyfile:
        return segyio.tools.cube(segyfile)

seismic = segy2numpy(filename_pp)
ai = segy2numpy(filename_ai)
fault = segy2numpy(filename_fault)
# f"Number of inlines: {seismic.shape[0]}, crosslines: {seismic.shape[1]}, samples: {seismic.shape[2]}"

In [5]:
def preprocess_seismic_data(seismic, fault):
    """
    Normalize seismic data and ensure faults are binary.
    """
    seismic = (seismic - np.min(seismic)) / (np.max(seismic) - np.min(seismic))
    fault = (fault > 0).astype(float)
    return seismic, fault

In [6]:

# --- Step 2: Dataset ---
class SeismicDataset(Dataset):
    def __init__(self, seismic, fault):
        self.seismic = seismic
        self.fault = fault

    def __len__(self):
        return self.seismic.shape[0]

    def __getitem__(self, idx):
        seismic_sample = self.seismic[idx]
        fault_sample = self.fault[idx]
        return (
            torch.tensor(seismic_sample, dtype=torch.float32).unsqueeze(0),
            torch.tensor(fault_sample, dtype=torch.float32).unsqueeze(0),
        )

In [7]:
# --- Step 3: UNet Model ---
class UNetWithFeatureExtraction(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNetWithFeatureExtraction, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, out_channels, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

In [8]:
import torch.nn.functional as F

# --- Updated Train UNet Function ---
def train_unet(model, train_loader, optimizer, criterion, device, n_epochs=10):
    model.train()
    for epoch in range(n_epochs):
        epoch_loss = 0
        for seismic, fault in train_loader:
            seismic, fault = seismic.to(device), fault.to(device)

            optimizer.zero_grad()
            outputs, _ = model(seismic)  # Get segmentation and features

            # Adjust target size to match model output
            pad_h = outputs.shape[2] - fault.shape[2]
            pad_w = outputs.shape[3] - fault.shape[3]
            fault = F.pad(fault, (0, pad_w, 0, pad_h))

            loss = criterion(outputs, fault)

            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {epoch_loss / len(train_loader):.4f}")


In [9]:
def extract_features_and_labels(seismic, fault):
    """
    Extracts features and labels with consistent lengths for SVM.
    Features are flattened seismic samples, and labels are corresponding binary fault indicators.
    """
    # Ensure spatial dimensions match
    if seismic.shape != fault.shape:
        raise ValueError(f"Seismic and fault dimensions do not match: {seismic.shape} vs {fault.shape}")

    # Flatten spatial dimensions for pixel-wise classification
    features = seismic.reshape(-1, seismic.shape[-1])  # Shape: (num_samples * height * width, depth)
    labels = fault.flatten()  # Shape: (num_samples * height * width, )

    # Ensure feature-label consistency
    if len(features) != len(labels):
        raise ValueError(f"Features and labels have inconsistent lengths: {len(features)} vs {len(labels)}")

    return features, labels


In [10]:
from sklearn.preprocessing import StandardScaler

def preprocess_for_svm(features, labels, threshold=0.5):
    """
    Preprocesses features by scaling and converts labels to binary format.
    """
    # Scale features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # Binarize labels
    labels_binary = (labels > threshold).astype(int)

    # Ensure feature-label consistency
    if len(features_scaled) != len(labels_binary):
        raise ValueError(f"Features and labels have inconsistent lengths: {len(features_scaled)} vs {len(labels_binary)}")

    return features_scaled, labels_binary, scaler


In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

def train_svm(features, labels):
    """
    Trains an SVM classifier on the extracted features and binary labels.
    """
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Train SVM
    svm_model = SVC(kernel="rbf", C=1, gamma="scale")
    svm_model.fit(X_train, y_train)

    # Predict on test set
    y_pred = svm_model.predict(X_test)

    return svm_model, X_test, y_test, y_pred


In [12]:
def visualize_results(test_seismic, test_fault, predictions):
    """
    Visualizes seismic data, fault ground truth, and predicted faults.
    """
    for i in range(min(3, test_seismic.shape[0])):  # Visualize up to 3 examples
        plt.figure(figsize=(12, 4))

        # Original Seismic Data
        plt.subplot(1, 3, 1)
        plt.title("Seismic Data")
        plt.imshow(test_seismic[i].T, cmap="seismic")
        plt.colorbar()

        # Ground Truth Fault
        plt.subplot(1, 3, 2)
        plt.title("Ground Truth Fault")
        plt.imshow(test_fault[i].T, cmap="gray")
        plt.colorbar()

        # Predicted Fault
        plt.subplot(1, 3, 3)
        plt.title("Predicted Fault")
        plt.imshow(predictions[i].reshape(test_fault[i].shape).T, cmap="gray")
        plt.colorbar()

        plt.show()


In [13]:
seismic, fault = preprocess_seismic_data(seismic, fault)

# Split data
train_seismic, test_seismic, train_fault, test_fault = train_test_split(
    seismic, fault, test_size=0.2, random_state=42
)

# Create datasets and loaders
train_dataset = SeismicDataset(train_seismic, train_fault)
test_dataset = SeismicDataset(test_seismic, test_fault)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Train UNet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNetWithFeatureExtraction().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_unet(model, train_loader, optimizer, criterion, device, n_epochs=10)



Epoch 1/10, Loss: 0.5132
Epoch 2/10, Loss: 0.1042
Epoch 3/10, Loss: 0.1208
Epoch 4/10, Loss: 0.0843
Epoch 5/10, Loss: 0.0802
Epoch 6/10, Loss: 0.0764
Epoch 7/10, Loss: 0.0758
Epoch 8/10, Loss: 0.0747
Epoch 9/10, Loss: 0.0743
Epoch 10/10, Loss: 0.0741


In [14]:
# Extract features for SVM
features, labels = extract_features_and_labels(seismic, fault)

ValueError: Features and labels have inconsistent lengths: 59489 vs 44676239

In [ ]:
# Preprocess features for SVM
features_scaled, labels_binary, scaler = preprocess_for_svm(features, labels, threshold=0.5)


In [ ]:
# Train SVM
svm_model, X_test, y_test, y_pred = train_svm(features_scaled, labels_binary)

In [ ]:

# Visualize results
visualize_results(test_seismic, test_fault, y_pred.reshape(test_fault.shape))